In [1]:
# import sys
# !{sys.executable} -m pip install selenium
# !{sys.executable} -m pip install html2text
# !{sys.executable} -m pip install lxml

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

import warnings
warnings.filterwarnings('ignore') # отключение вывода предупреждений

In [3]:
# настраиваемые параметры
chrome_driver_path = "chromedriver.exe" # путь до исполняемого файла (для скачивания) 
links_filename = "links.txt" # путь до файла с временным сохранением ссылок
article_filename = "articles.json" # путь до файла с полным сохранением статей
categories = ["entertainment", "auto", "sport", "health"] # искомые категории дла сайта nn.ru (развлечения, авто, спорт, здоровье)
article_count = 1000 # число статей по каждой теме (минимум)

In [4]:
def parse_nn(categories, article_count, output): # categories - искомые категории, article_count - сколько будет искаться статей по кажой теме, output - куда записывать результат
    prefs = {"profile.managed_default_content_settings.images": 2,
             "profile.default_content_settings.cookies": 2} # дополнительные настройки для поиска?
    chrome_options = Options() # handler для опций поиска
    chrome_options.add_extension('adblock.crx') # добавление расширения
    chrome_options.add_experimental_option("prefs",prefs) # добавление заготовленных опций
    chrome_options.add_argument('--disable-application-cache') # отключение кэширования посещённой страницы
    
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options) # инициализация Chrome WebDriver
    driver.implicitly_wait(10) # ожидание прогрузки
    time.sleep(10)
    driver.close() # закрытие выбранной страницы браузера
#     print("Current session is {}".format(driver.session_id)) # вывод id сессии
    driver.switch_to.window(driver.window_handles[0]) # смена на новое окно
    driver.get('chrome-extension://gighmmpiobklfepjocnamgkkbiglidom/options.html') # открытие страницы
    driver.find_element("id", 'acceptable_ads').click() # поиск элемента по id + нажатие на него (adblock customization)

# URL example "https://www.nn.ru/text/?rubric=entertainment&page=6"
    with open(output, 'a') as file: # открываем файл для дозаписи
        for category in categories:
            page = 1 # с какой страницы начнём
            counter = 0 # зануляем счётчик скаченных статей
            while counter < article_count:
                driver.get(f"https://www.nn.ru/text/?rubric={category}&page={page}") # итеративно переходим на стриницы
                for element in tqdm.tqdm(driver.find_elements("xpath", "//article"),position=0): # цикл по полученным элементам "article" на странице (ищем блоки html <article>), xpath с // ищет блоки html любой вложенности (указан не полный путь до html блока)
                    addr = element.find_element("tag name", 'a').get_attribute('href')
                    if addr.find(f"https://www.nn.ru/text/{category}/") != -1: # проверяем ссылку на нужный нам вид
                        file.write(json.dumps({ # генерирует и записываем json (link + category)
                            'link' : addr,
                            'category': category
                        },ensure_ascii=False) + '\n') # без трансформации символов в unicode
                        counter += 1
                page += 1
    driver.quit() # закрытие браузера и всех вкладок

In [5]:
parse_nn(categories, article_count, links_filename)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 32.99it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 29.37it/s]


In [6]:
posts = []
with open(links_filename) as file: # открываем файл
    for line in file: # проходим по всем строкам
        posts.append(json.loads(line)) # парсим и добавляем строку из джейсона в posts

In [7]:
for element in tqdm.tqdm(posts): # проходим по всем постам (ссылка + категория)
    soup = BeautifulSoup(requests.get(element['link']).text, "lxml") # передача ответа от сервера для парсинга lxml
# requests.get(element['link']).text - контент, что вернул сервер после запроса
# Объект BeautifulSoup() представляет HTML-документ как единое целое.
    text = ""
    tags = []

    for paragraph in soup.findAll('p', attrs={'class': ''}): # поиск текста на странице в блоке <p></p> (soup.findAll просматривает и извлекает ВСЕХ потомков тега, которые соответствуют переданным фильтрующим аргументам)
        text += html2text(paragraph.text).replace('\n', ' ') + ' ' # конвертация в текст
    
    for tag in soup.findAll('a', attrs={'class': ['R0qz0 NJL+l M5IeQ', 'ZaoKY Hz+BA']}):
        tags.append(html2text(tag.text).replace('\n', ' ').strip())
    
#     print('article_id ', element['link'], '\n',
#           'title ', soup.h1.text,'\n',
#           'category ', element['category'],'\n',
#           'tags ', tags,'\n',
#           'text ', text)
    
    with open(article_filename, 'a', encoding="utf-8") as file: # запись в файл всех данных
        file.write(json.dumps({
            'article_id': element['link'],
            'title' : soup.h1.text,
            'category': element['category'],
            'tags': ','.join(tags),
            'text': text,
        },ensure_ascii=False) + '\n')

100%|██████████████████████████████████████████████████████████████████████████████| 4092/4092 [43:14<00:00,  1.58it/s]


In [8]:
# обзор датасета
news = []
with open(article_filename, encoding="utf-8") as file: # открываем файл
    for line in file: # проходим по всем строкам
        news.append(json.loads(line)) # парсим и добавляем строку из джейсона в news
data = pd.DataFrame(news) # создаём датафрейм
data.head()

,article_id,title,category,tags,text
0,https://www.nn.ru/text/entertainment/2023/01/1...,Вам не решить: 10 советских задач в картинках ...,entertainment,"Развлечения,СССР,Интересный тест,Тест,Внимание...",Ставить опыты над возможностями человеческих п...
1,https://www.nn.ru/text/entertainment/2023/01/1...,Не только «Ёлки-9»: что больше всего приглянул...,entertainment,Развлечения,Wink представляет самые популярные фильмы и се...
2,https://www.nn.ru/text/entertainment/2023/01/0...,В феврале выйдет сериал о самом жутком московс...,entertainment,Развлечения,Видеосервисы Wink и more.tv представили первые...
3,https://www.nn.ru/text/entertainment/2023/01/0...,Это что еще за амишка! Разберетесь в молодежно...,entertainment,"Развлечения,Жаргон,Сленг,Слова,Тест",«Раньше было лучше» — всего лишь наше убеждени...
4,https://www.nn.ru/text/entertainment/2023/01/0...,"«Я этого не скрываю, по моим фото в соцсетях в...",entertainment,"Развлечения,Аниме,Косплей,Учитель",Учитель русского языка и литературы Ксения Пло...
